En este Jupyter Notebook, abordaremos la creación de diversos conjuntos de datos fundamentales para el funcionamiento de nuestra API. El enfoque principal en esta etapa es optimizar y consolidar estos conjuntos de datos, lo que resultará en una mayor eficiencia en los cálculos y evitará problemas de renderización debido a las restricciones de memoria en su entorno. Para abordar la limitada capacidad de almacenamiento proporcionada por Render, hemos decidido cargar y utilizar estos conjuntos de datos de manera más eficiente, en lugar de cargar los datos completos.

In [156]:
import warnings
warnings.filterwarnings("ignore")



# Inicialización

Importamos las librerías que nos servirán para poder procesar, visualizar y explorar nuestros datos de manera efectiva, la librería `pandas`, `numpy` `pyarrow`.

In [157]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extracción de los datos

Para realizar los análisis requeridos, extraeremos los datos esenciales de los conjuntos `data_games`, `data_reviews`, y `data_items`. Estos datos nos permitirán realizar los análisis requeridos.

In [158]:
# extraccion de los datos de data_games
data_games = pd.read_csv('data_games_cleaned.csv')

In [159]:
# extraccion de los datos de data_reviews (analisis de sentimiento)
data_reviews = pd.read_csv('data_reviews_sentiment_analysis.csv')

In [160]:
# extraccion de los datos de data_items
data_items = pd.read_csv('data_items_cleaned.csv')

# PlayTimeGenre(genero: str):

Esta función tiene como objetivo encontrar el año en el que se registraron las mayores horas jugadas por los usuarios en un género de videojuego específico. Para lograrlo, procederemos a construir un DataFrame auxiliar, que nos permitirá realizar los cálculos y análisis de manera más eficiente y precisa.

Comenzamos tomando las columnas necesarias de los DataFrames data_games y data_items. Estas columnas específicas son esenciales para nuestro análisis, lo que nos permitirá enfocarnos en los datos más relevantes y mejorar la eficiencia del proceso.

In [161]:
# Extraer las columnas necesarias de data_games y data_items
filtered_games_data = data_games[['id', 'genres','release_year']]
items_data = data_items[['user_id', 'item_id', 'playtime_forever']]


In [162]:
filtered_games_data.head(3)

,id,genres,release_year
0,761140,Action,2018
1,761140,Casual,2018
2,761140,Indie,2018


In [163]:
items_data.head(3)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7


**Combinación de Datos:** Iniciamos uniendo los datos de los juegos y los datos de los ítems, utilizando 'item_id' e 'id' como claves, para obtener información completa sobre los juegos y los jugadores.

In [164]:
# Unir los datos de data_games y data_items usando 'item_id' ,'id' como clave
combined_data = pd.merge(items_data, filtered_games_data, left_on='item_id', right_on='id')


In [165]:
combined_data

,user_id,item_id,playtime_forever,id,genres,release_year
0,76561197970982479,10,6,10,Action,2000
1,js41637,10,0,10,Action,2000
2,Riot-Punch,10,0,10,Action,2000
3,doctr,10,93,10,Action,2000
4,corrupted_soul,10,108,10,Action,2000
...,...,...,...,...,...,...
9877299,76561198107283457,354280,164,354280,Indie,2016
9877300,76561198107283457,354280,164,354280,Simulation,2016
9877301,inven,433920,0,433920,Adventure,2016
9877302,inven,433920,0,433920,Indie,2016


**Agrupación por Género y Año:** Luego, agrupamos los datos combinados por género y año y calculamos la suma de las horas jugadas en cada combinación de género y año.

In [166]:
# Agrupar por género, año y sumar las horas jugadas
grouped_data = combined_data.groupby(['genres', 'release_year'])['playtime_forever'].sum().reset_index()


In [167]:
grouped_data

,genres,release_year,playtime_forever
0,Action,1983,3473
1,Action,1984,384
2,Action,1988,16001
3,Action,1989,607
4,Action,1990,18335
...,...,...,...
349,Web Publishing,2013,333678
350,Web Publishing,2014,33641
351,Web Publishing,2015,348673
352,Web Publishing,2016,136


In [168]:
grouped_data['genres'].unique()

array(['Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing',
       'Simulation', 'Software Training', 'Sports', 'Strategy',
       'Utilities', 'Video Production', 'Web Publishing'], dtype=object)

Identificamos el año de mayor popularidad para cada género, basándonos en la cantidad de horas jugadas, y luego ordenamos los géneros según las horas jugadas en su año más destacado, lo que nos proporciona una lista de géneros junto con su año más relevante y la cantidad de horas jugadas en ese período.

In [169]:
# Encuentra el año en que se jugaron más horas para cada género
most_played_year = grouped_data.loc[grouped_data.groupby('genres')['playtime_forever'].idxmax()]

# Ordena los géneros por la cantidad de horas jugadas
most_played_year = most_played_year.sort_values(by='playtime_forever', ascending=False)

In [170]:
most_played_year

,genres,release_year,playtime_forever
26,Action,2012,1085635110
162,Indie,2006,446115272
264,Simulation,2006,444536391
213,RPG,2011,227959458
60,Adventure,2011,221707756
145,Free to Play,2013,146213734
323,Strategy,2013,145266750
183,Massively Multiplayer,2013,133376503
119,Early Access,2013,118902893
106,Casual,2015,81708365


El resultado final es un DataFrame que muestra los géneros de videojuegos junto con el año en que se jugaron más horas y la cantidad total de horas jugadas en ese año. Esto proporciona una visión general de cuándo los jugadores pasaron más tiempo jugando videojuegos de un género específico y facilita su posterior uso en el despliegue.

Luego lo guardamos en formato CSV para facilitar su manejo en la construcción de nuestra función PlayTimeGenre(genero: str). También aprovechamos este punto para guardar los DataFrames en formato Parquet, para optimizar la estructura de los datos en el deploy.

In [171]:
# Guardar en CSV:
most_played_year.to_csv('df_most_played_year.csv', index=False, encoding='utf-8')
print(f'Se ha guardado el archivo df_most_played_year.csv en la misma carpeta.')


Se ha guardado el archivo df_most_played_year.csv en la misma carpeta.


In [172]:
# Guardar en Parquet:
# Convierte el DataFrame a una tabla de PyArrow
table = pa.Table.from_pandas(most_played_year)
pq.write_table(table, 'df_most_played_year.parquet')
print(f'Se ha guardado el archivo df_most_played_year.parquet en la misma carpeta.')

Se ha guardado el archivo df_most_played_year.parquet en la misma carpeta.


# UserForGenre( genero : str ):

Antes de comenzar a implementar la función UserForGenre(genero: str), podemos crear un DataFrame para facilitar todo el proceso, el DataFrame muestra el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Extraer las columnas necesarias de data_reviews y data_items.

In [218]:
# Extraer las columnas de data_reviews
data_reviews_subset = data_reviews[['reviews_item_id','user_id','reviews_year']]
data_reviews_subset = data_reviews_subset.rename(columns={'reviews_item_id': 'item_id'})

# Eliminar filas con 'reviews_year' igual a 'Dato no disponible'
data_reviews_subset = data_reviews_subset[data_reviews_subset['reviews_year'] != 'Dato no disponible']

# Extrae las columnas necesarias de data_items
data_items_subset = data_items[['user_id', 'item_id', 'playtime_forever']]



In [214]:
data_reviews_subset.head(3)

,item_id,user_id,reviews_year
0,1250,76561197970982479,2011
1,251610,js41637,2014
3,250320,doctr,2013


In [215]:
data_items_subset.head(3)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7


Procedemos a fusionar conjuntos de datos clave para nuestro análisis. Utilizando las columnas 'user_id', 'item_id'  como puntos de referencia, llevamos a cabo la unión de los datos de interés.

In [176]:
# Unir data_items_subset y data_reviews_subset utilizando 'user_id', 'item_id'como claves
merged_data = data_items_subset.merge(data_reviews_subset, on=["user_id", "item_id"], how="inner")

In [177]:
merged_data

,user_id,item_id,playtime_forever,reviews_year
0,76561197970982479,22200,271,2011
1,76561197970982479,1250,10006,2011
2,76561197970982479,43110,834,2011
3,js41637,227300,551,2013
4,js41637,239030,349,2013
...,...,...,...,...
36096,wayfeng,730,42740,2015
36097,76561198251004808,253980,1098,2015
36098,72947282842,730,33,2015
36099,ApxLGhost,730,10121,2015


Utilizamos las columnas 'item_id' y 'id' como puntos de referencia para combinar la información detallada contenida en los conjuntos de datos "merged_data" y "data_games".

In [178]:
# Unir merged_data y data_games usando 'item_id' y 'id' como claves
final_merged_data = merged_data.merge(data_games[['id', 'genres']], left_on='item_id', right_on='id', how='inner')


In [179]:
final_merged_data

,user_id,item_id,playtime_forever,reviews_year,id,genres
0,76561197970982479,22200,271,2011,22200,Action
1,76561197970982479,22200,271,2011,22200,Indie
2,pipekissXD,22200,690,2013,22200,Action
3,pipekissXD,22200,690,2013,22200,Indie
4,ClockworkLunatic,22200,23,2015,22200,Action
...,...,...,...,...,...,...
78969,evilindiegaming,367780,107,2015,367780,Action
78970,evilindiegaming,367780,107,2015,367780,Indie
78971,llDracuwulf,307130,211,2015,307130,Action
78972,llDracuwulf,307130,211,2015,307130,Adventure


Nos enfocamos en la agregación y organización de datos cruciales. Agrupamos la información por usuario, año y género, permitiéndonos examinar la acumulación de horas jugadas en cada categoría. Luego, ordenamos estos datos de mayor a menor, lo que nos brinda una visión más clara y facilita la identificación de los usuarios más activos en términos de horas de juego. 

In [216]:
# Agrupar por usuario (user_id), año (reviews_year) y género, sumar las horas jugadas
aggregated_data = final_merged_data.groupby(['user_id', 'reviews_year', 'genres'])['playtime_forever'].sum().reset_index()

# Ordenar de mayor a menor por horas jugadas
sorted_data = aggregated_data.sort_values(by='playtime_forever', ascending=False)

# Convertir 'playtime_forever' de minutos a horas
sorted_data['playtime_forever'] = sorted_data['playtime_forever'] / 60

In [217]:
sorted_data.head(20)

,user_id,reviews_year,genres,playtime_forever
64114,wolop,2011,Simulation,10712.883333
64113,wolop,2011,Indie,10712.883333
5380,76561198039832932,2015,Indie,10283.900000
5382,76561198039832932,2015,Simulation,10232.350000
60502,shinomegami,2015,Free to Play,10183.450000
63161,tsunamitad,2015,Indie,10001.133333
63162,tsunamitad,2015,Simulation,10001.133333
63160,tsunamitad,2015,Casual,10001.133333
60504,shinomegami,2015,Massively Multiplayer,8848.033333
60499,shinomegami,2015,Action,8848.033333


Como parte de nuestro previo Análisis Exploratorio de Datos (EDA), habíamos observado registros particulares que presentaban una acumulación de horas de juego que superaban ampliamente las 8760 horas, equivalente a un año completo. Esta situación excepcional estaba fuera del rango de posibilidad realista y podría influir en la integridad de nuestros resultados y análisis. Por lo tanto, hemos tomado la decisión de eliminar este registro específico, asegurando que nuestra base de datos refleje datos más coherentes y representativos.

In [183]:
# Eliminar filas donde 'playtime_forever' es mayor a 8760
sorted_data = sorted_data[sorted_data['playtime_forever'] <= 8760]

Luego, calculamos la suma de las horas jugadas ('playtime_forever') para cada grupo. Esto nos proporciona la cantidad total de horas que cada usuario ha jugado en juegos de un género específico durante un año en particular.

In [185]:
# Agrupar por 'genres' y 'user_id', sumar las horas jugadas
aggregated_data_by_genre = sorted_data.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()


In [186]:
aggregated_data_by_genre

,genres,user_id,playtime_forever
0,Action,--000--,49.150000
1,Action,--ace--,21.150000
2,Action,--ionex--,14.033333
3,Action,-2SV-vuLB-Kg,515.916667
4,Action,-Beave-,47.733333
...,...,...,...
57061,Web Publishing,draoftwmate,0.950000
57062,Web Publishing,joshfeb06,5.066667
57063,Web Publishing,odoroitaka,20.733333
57064,Web Publishing,thetimegoesfastxd77,67.950000


En nuestro análisis, hemos consolidado un primer DataFrame que destaca a usuarios individuales, uno por género, quienes han acumulado la mayor cantidad de horas de juego en ese género específico. 

In [187]:
# Luego, encontramos al usuario que más horas acumuló en cada género
user_max_hours = aggregated_data_by_genre.groupby('genres')['user_id', 'playtime_forever'].apply(lambda x: x.loc[x['playtime_forever'].idxmax()]).reset_index()

In [188]:
user_max_hours

,genres,user_id,playtime_forever
0,Action,thiefofrosesinlalaland,7438.700000
1,Adventure,thiefofrosesinlalaland,6776.366667
2,Animation &amp; Modeling,76561198059330972,1090.450000
3,Audio Production,SambaWarKiddo,117.083333
4,Casual,76561197972452208,3466.900000
5,Design &amp; Illustration,Xyphien,1183.516667
6,Early Access,mutatedwombat,4530.950000
7,Education,76561198059330972,1090.450000
8,Free to Play,thiefofrosesinlalaland,6159.950000
9,Indie,ThisIsWhereIGetOff,8344.950000


Procedemos al armado del otro DataFrame para mostrar las horas jugadas por año en cada género para cada usuario.

partimos de nuestro DataFrame aggregated_data para el armado de este nuevo DataFrame

In [209]:
aggregated_data

,user_id,reviews_year,genres,playtime_forever
0,--000--,2014,Action,49.15
1,--ace--,2014,Action,21.15
2,--ace--,2014,Adventure,21.15
3,--ace--,2014,Indie,21.15
4,--ace--,2014,RPG,21.15
...,...,...,...,...
64988,zyr0n1c,2014,Action,10.72
64989,zyr0n1c,2014,Free to Play,10.72
64990,zyr0n1c,2014,Indie,10.72
64991,zyr0n1c,2015,Action,1112.28


Realizamos dos transformaciones importantes en nuestros datos. En primer lugar, hemos asegurado que la variable 'reviews_year' esté representada como un tipo de dato entero (int), lo que facilitará futuros cálculos y comparaciones basadas en el año de revisión. En segundo lugar, hemos convertido 'playtime_forever' de minutos a horas, una conversión esencial para obtener una comprensión más precisa del tiempo dedicado por los usuarios a sus actividades de juego. 

In [194]:

# Asegurarse de que 'reviews_year' sea de tipo int
aggregated_data['reviews_year'] = aggregated_data['reviews_year'].astype(int)

# Convertir 'playtime_forever' de minutos a horas
aggregated_data['playtime_forever'] = (aggregated_data['playtime_forever'] / 60).round(2)



Llevamos a cabo una operación de pivote en nuestros datos, utilizando 'user_id' y 'genres' como índices clave. Esto nos ha permitido transformar nuestros datos en una forma más organizada y tabular, donde cada fila representa un usuario y su tiempo de juego en géneros específicos a lo largo de los años. 

In [205]:
# Pivota los datos y agrega las horas acumuladas utilizando como índices 'user_id' y 'genres'
pivot_data = aggregated_data.pivot_table(index=['user_id', 'genres'], columns='reviews_year', values='playtime_forever', aggfunc='sum', fill_value=0)



In [208]:
pivot_data = pivot_data.reset_index()


In [220]:
pivot_data.head(3)

reviews_year,user_id,genres,2010,2011,2012,2013,2014,2015
0,--000--,Action,0.0,0.0,0.0,0.0,49.15,0.0
1,--ace--,Action,0.0,0.0,0.0,0.0,21.15,0.0
2,--ace--,Adventure,0.0,0.0,0.0,0.0,21.15,0.0


Fusionamos dos conjuntos de datos clave. El primer conjunto, denominado 'user_max_hours', contiene información sobre los usuarios que han acumulado la mayor cantidad de horas en géneros específicos. El segundo conjunto, 'pivot_data', ha sido sometido a una operación de pivote para mostrar las horas jugadas por año en cada género para cada usuario. Al combinar estos dos conjuntos de datos utilizando las columnas 'genres' y 'user_id' como claves, hemos creado un nuevo DataFrame llamado 'result_df'.

In [221]:
df_hours_year_cleaned = pd.merge(user_max_hours, pivot_data, on=['genres', 'user_id'], how='inner')


In [222]:
df_hours_year_cleaned

,genres,user_id,playtime_forever,2010,2011,2012,2013,2014,2015
0,Action,thiefofrosesinlalaland,7438.700000,0.0,0.0,0.0,0.0,7107.58,331.12
1,Adventure,thiefofrosesinlalaland,6776.366667,0.0,0.0,0.0,0.0,6445.25,331.12
2,Animation &amp; Modeling,76561198059330972,1090.450000,0.0,0.0,0.0,0.0,0.00,1090.45
3,Audio Production,SambaWarKiddo,117.083333,0.0,0.0,0.0,0.0,117.08,0.00
4,Casual,76561197972452208,3466.900000,0.0,0.0,0.0,0.0,0.00,3466.90
5,Design &amp; Illustration,Xyphien,1183.516667,0.0,0.0,0.0,0.0,0.00,1183.52
6,Early Access,mutatedwombat,4530.950000,0.0,0.0,0.0,0.0,0.00,4530.95
7,Education,76561198059330972,1090.450000,0.0,0.0,0.0,0.0,0.00,1090.45
8,Free to Play,thiefofrosesinlalaland,6159.950000,0.0,0.0,0.0,0.0,6159.95,0.00
9,Indie,ThisIsWhereIGetOff,8344.950000,0.0,0.0,0.0,0.0,8250.97,93.98


Nuestra data df_hours_year_cleaned contiene la información relacionada con el usuario que acumula más horas jugadas para un género de videojuegos específico. Además, muestra una lista de la acumulación de horas jugadas por año para ese género en particular.

Lo guardamos en formato CSV para facilitar su manejo en la construcción de nuestra función UserForGenre(genero: str). También aprovechamos este punto para guardar los DataFrames en formato Parquet, para optimizar la estructura de los datos en el deploy.

In [223]:
# Guardar en CSV:
df_hours_year_cleaned.to_csv('df_maxuser_genre_cleaned.csv', index=False, encoding='utf-8')
print(f'Se ha guardado el archivo df_maxuser_genre_cleaned.csv en la misma carpeta.')

Se ha guardado el archivo df_maxuser_genre_cleaned.csv en la misma carpeta.


In [224]:
# Guardar en Parquet:
# Convierte el DataFrame a una tabla de PyArrow
table = pa.Table.from_pandas(df_hours_year_cleaned)
pq.write_table(table, 'df_maxuser_genre_cleaned.parquet')
print(f'Se ha guardado el archivo df_maxuser_genre_cleaned.parquet en la misma carpeta.')

Se ha guardado el archivo df_maxuser_genre_cleaned.parquet en la misma carpeta.


# UsersRecommend( año : int ):

crearemos un DataFrame de las tres recomendaciones principales por año de los usuarios. A través de este proceso, esperamos arrojar luz sobre cómo los jugadores expresan sus preferencias y proporcionan una guía valiosa para otros entusiastas de los videojuegos.

Comenzamos tomando las columnas necesarias de los DataFrames data_games y data_reviews. Estas columnas específicas son esenciales para nuestro análisis, lo que nos permitirá enfocarnos en los datos más relevantes y mejorar la eficiencia del proceso.

In [227]:
# Extraer las columnas necesarias de data_reviews
df_reviews_aux = data_reviews[['reviews_item_id','reviews_recommend','reviews_year']]
# Cambiamos el nombre a la columna 'reviews_item_id' por 'id' 
df_reviews_aux = df_reviews_aux.rename(columns={'reviews_item_id':'id'})


In [228]:
df_reviews_aux

,id,reviews_recommend,reviews_year
0,1250,True,2011
1,251610,True,2014
2,248820,True,Dato no disponible
3,250320,True,2013
4,211420,True,2014
...,...,...,...
58164,440,True,2014
58165,304930,True,2014
58166,265630,True,2015
58167,304050,True,2015


In [229]:
# Extraer las columnas necesarias de data_games
df_games_aux = data_games[['id','app_name']]
df_games_aux = df_games_aux.drop_duplicates()

In [230]:
df_games_aux

,id,app_name
0,761140,Lost Summoner Kitty
5,643980,Ironbound
9,670290,Real Pool 3D - Poolians
14,767400,弹炸人2222
17,772540,Battle Royale Trainer
...,...,...
71535,745400,Kebab it Up!
71539,773640,Colony On Mars
71543,733530,LOGistICAL: South Africa
71546,610660,Russian Roads


Fusionamos los dos DataFrames, df_reviews_aux y df_games_aux, utilizando la columna 'id' como clave común para combinarla información de reseñas y juegos en un solo DataFrame.

In [231]:
df_reviews_aux = df_reviews_aux.merge(df_games_aux, on='id')

In [232]:
df_reviews_aux

,id,reviews_recommend,reviews_year,app_name
0,1250,True,2011,Killing Floor
1,1250,True,2015,Killing Floor
2,1250,True,2014,Killing Floor
3,1250,True,2013,Killing Floor
4,1250,True,2014,Killing Floor
...,...,...,...,...
49482,73010,True,2012,Cities in Motion
49483,378930,False,Dato no disponible,Pesadelo - Regressão
49484,16600,True,2012,Trials 2: Second Edition
49485,232950,True,2014,Bridge Project


En este proceso, analizamos las reseñas de los usuarios para identificar los juegos mejor recomendados por año. Primero, filtramos el DataFrame para seleccionar solo las reseñas con recomendaciones positivas. Luego, agrupamos las reseñas por año y juego, contabilizando las recomendaciones. Posteriormente, ordenamos esta información en orden descendente para obtener los juegos más recomendados por año. Finalmente, agregamos una columna que muestra la posición de cada juego en el ranking de recomendaciones por año.

In [233]:
# Filtrar el DataFrame para quedarse solo con las recomendaciones positivas (reviews_recommend == True)
df_positive_reviews = df_reviews_aux[df_reviews_aux['reviews_recommend'] == True]

# Agrupar por año y juego, y contar las recomendaciones positivas
grouped = df_positive_reviews.groupby(['reviews_year', 'app_name'])['reviews_recommend'].sum().reset_index()

# Ordenar en orden descendente por recomendaciones
sorted_grouped = grouped.sort_values(by=['reviews_year', 'reviews_recommend'], ascending=[True, False])

# Seleccionar los juegos más recomendados por año
top_positive_games_by_year = sorted_grouped.groupby('reviews_year').head(3)

# Agregar una columna con la posición en el ranking
top_positive_games_by_year['rank'] = top_positive_games_by_year.groupby('reviews_year')['reviews_recommend'].rank(ascending=False, method='first')


In [234]:
top_positive_games_by_year

,reviews_year,app_name,reviews_recommend,rank
30,2010,Team Fortress 2,10,1.0
13,2010,Killing Floor,6,2.0
0,2010,Alien Swarm,4,3.0
174,2011,Team Fortress 2,77,1.0
142,2011,Portal 2,26,2.0
175,2011,Terraria,23,3.0
385,2012,Team Fortress 2,262,1.0
387,2012,Terraria,42,2.0
275,2012,Garry's Mod,38,3.0
910,2013,Team Fortress 2,786,1.0


In [239]:
# Realizar una operación de pivote en el DataFrame
pivot_df = top_positive_games_by_year.pivot(index='reviews_year', columns='rank', values='app_name')

# Renombrar las columnas según tus requerimientos
pivot_df.columns = ['Rank 1', 'Rank 2', 'Rank 3']

# Restablecer el índice para tener 'reviews_year' como una columna
pivot_df.reset_index(inplace=True)

# Agregar la columna 'reviews_recommend' para tener la información de recomendaciones
pivot_df['Reviews Recommend'] = top_positive_games_by_year.groupby('reviews_year')['reviews_recommend'].max().values

# Reordenar las columnas según tus requerimientos
pivot_df = pivot_df[['reviews_year', 'Rank 1', 'Rank 2', 'Rank 3']]

In [243]:
pivot_df

,reviews_year,Rank 1,Rank 2,Rank 3
0,2010,Team Fortress 2,Killing Floor,Alien Swarm
1,2011,Team Fortress 2,Portal 2,Terraria
2,2012,Team Fortress 2,Terraria,Garry's Mod
3,2013,Team Fortress 2,Garry's Mod,Left 4 Dead 2
4,2014,Team Fortress 2,Counter-Strike: Global Offensive,Garry's Mod
5,2015,Counter-Strike: Global Offensive,Team Fortress 2,Garry's Mod
6,Dato no disponible,Counter-Strike: Global Offensive,Team Fortress 2,Garry's Mod


Hemos obtenido un DataFrame, top_positive_games_by_year, que muestra los juegos más recomendados por año, junto con el número de recomendaciones y su clasificación en el ranking. Este análisis proporciona información valiosa sobre los juegos más populares en cada año.

A continuación, procedemos a guardar este DataFrame en formato CSV y Parquet para facilitar su manejo y análisis en el proyecto.

In [244]:
# Guardar en CSV
pivot_df.to_csv('df_top_positive_games_by_year.csv', index=False, encoding='utf-8')
print(f'Se ha guardado el archivo df_top_positive_games_by_year.csv en la misma carpeta.')

Se ha guardado el archivo df_top_positive_games_by_year.csv en la misma carpeta.


In [245]:
# Guardar en Parquet
table = pa.Table.from_pandas(pivot_df)
pq.write_table(table, 'df_top_positive_games_by_year.parquet')
print(f'Se ha guardado el archivo df_top_positive_games_by_year.parquet en la misma carpeta.')

Se ha guardado el archivo df_top_positive_games_by_year.parquet en la misma carpeta.


# UsersNotRecommend( año : int ): 

Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.

Al igual que nuestro análisis anterior de recomendaciones en la comunidad de jugadores, nos enfocaremos ahora en identificar los juegos menos recomendados por los usuarios en cada año.

Analizamos las reseñas de los usuarios para identificar los juegos menos recomendados por año. Primero, filtramos el DataFrame para seleccionar solo las reseñas con recomendaciones negativas. Luego, agrupamos las reseñas por año y juego, contabilizando las recomendaciones negativas. Posteriormente, ordenamos esta información en orden descendente para obtener los juegos menos recomendados por año. Finalmente, agregamos una columna que muestra la posición de cada juego en el ranking de `reviews_no_recommend` por año.

In [247]:
# Filtrar el DataFrame para quedarse solo con las recomendaciones negativas (reviews_recommend == False)
df_negative_reviews = df_reviews_aux[df_reviews_aux['reviews_recommend'] == False]

# Agrupar por año y juego, y contar las recomendaciones negativas
grouped = df_negative_reviews.groupby(['reviews_year', 'app_name'])['reviews_recommend'].count().reset_index()
grouped = grouped.rename(columns={'reviews_recommend': 'reviews_no_recommend'})

# Ordenar en orden descendente por recomendaciones negativas
sorted_grouped = grouped.sort_values(by=['reviews_year', 'reviews_no_recommend'], ascending=[True, False])

# Seleccionar los juegos con más recomendaciones negativas por año
top_negative_games_by_year = sorted_grouped.groupby('reviews_year').head(3)

# Agregar una columna con la posición en el ranking
top_negative_games_by_year['rank'] = top_negative_games_by_year.groupby('reviews_year')['reviews_no_recommend'].rank(ascending=False, method='first')




In [248]:
top_negative_games_by_year

,reviews_year,app_name,reviews_no_recommend,rank
0,2010,Team Fortress 2,1,1.0
1,2011,And Yet It Moves,2,1.0
5,2011,Team Fortress 2,2,2.0
2,2011,Counter-Strike: Source,1,3.0
6,2012,Aliens vs. Predator™,1,1.0
7,2012,Blacklight: Retribution,1,2.0
8,2012,Call of Duty®: Black Ops II,1,3.0
31,2013,Call of Duty®: Ghosts,14,1.0
114,2013,Team Fortress 2,12,2.0
17,2013,Ace of Spades: Battle Builder,9,3.0


In [257]:
# Realizar una operación de pivote en el DataFrame
pivot_neg_df = top_negative_games_by_year.pivot(index='reviews_year', columns='rank', values='app_name')

# Renombrar las columnas según tus requerimientos
pivot_neg_df.columns = ['Rank 1', 'Rank 2', 'Rank 3']

# Restablecer el índice para tener 'reviews_year' como una columna
pivot_neg_df.reset_index(inplace=True)

# Agregar la columna 'reviews_recommend' para tener la información de recomendaciones
pivot_neg_df['Reviews NO Recommend'] = top_negative_games_by_year.groupby('reviews_year')['reviews_no_recommend'].max().values

# Reordenar las columnas según tus requerimientos
pivot_neg_df = pivot_neg_df[['reviews_year', 'Rank 1', 'Rank 2', 'Rank 3']]

# Reemplazar NaN con "Sin Reviews Negativos"
pivot_neg_df = pivot_neg_df.replace({pd.NA: 'Sin Reviews Negativos'})

In [258]:
pivot_neg_df

,reviews_year,Rank 1,Rank 2,Rank 3
0,2010,Team Fortress 2,Sin Reviews Negativos,Sin Reviews Negativos
1,2011,And Yet It Moves,Team Fortress 2,Counter-Strike: Source
2,2012,Aliens vs. Predator™,Blacklight: Retribution,Call of Duty®: Black Ops II
3,2013,Call of Duty®: Ghosts,Team Fortress 2,Ace of Spades: Battle Builder
4,2014,DayZ,Counter-Strike: Global Offensive,Unturned
5,2015,Counter-Strike: Global Offensive,DayZ,Rust
6,Dato no disponible,No Man's Sky,Counter-Strike: Global Offensive,Team Fortress 2


Hemos identificado y compilado los juegos menos recomendados por la comunidad de jugadores para cada año.

Continuando con nuestro enfoque lo guardamos en formato CSV para facilitar su manejo en la construcción de nuestra función def UsersNotRecommend( año : int ). 
Asi como también para guardar los DataFrames en formato Parquet, para optimizar la estructura de los datos en el deploy.

In [259]:
# Guardar en CSV
pivot_neg_df.to_csv('df_top_negative_games_by_year.csv', index=False, encoding='utf-8')
print(f'Se ha guardado el archivo df_top_negative_games_by_year.csv en la misma carpeta.')

Se ha guardado el archivo df_top_negative_games_by_year.csv en la misma carpeta.


In [260]:
# Guardar en Parquet
table = pa.Table.from_pandas(pivot_neg_df)
pq.write_table(table, 'df_top_negative_games_by_year.parquet')
print(f'Se ha guardado el archivo df_top_negative_games_by_year.parquet en la misma carpeta.')

Se ha guardado el archivo df_top_negative_games_by_year.parquet en la misma carpeta.


# sentiment_analysis( año : int ): 

Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.


Extraemos las columnas relevantes de los conjuntos de datos. De data_reviews, tomaremos las columnas 'user_id', 'reviews_item_id', y 'sentiment_analysis', que contienen información sobre los usuarios, los juegos y el análisis de sentimiento de las reseñas. De data_games, extraeremos las columnas 'id' y 'release_year' para obtener el año de lanzamiento de los juegos, ya que es un factor crucial en nuestro análisis.

In [136]:
df_reviews_sentiment = data_reviews[['user_id','reviews_item_id','sentiment_analysis']]


In [137]:
df_reviews_sentiment

,user_id,reviews_item_id,sentiment_analysis
0,76561197970982479,1250,2
1,js41637,251610,2
2,evcentric,248820,2
3,doctr,250320,2
4,maplemage,211420,1
...,...,...,...
57392,How51,440,2
57393,76561198111410893,304930,2
57394,zaza147,265630,2
57395,lifeonhigh,304050,2


necesito ano de lanzamiento de data_games release_year

In [138]:
df_games_sentiment = data_games[['id','release_year']]
df_games_sentiment = df_games_sentiment.drop_duplicates()

In [139]:
df_games_sentiment

,id,release_year
0,761140,2018
5,643980,2018
9,670290,2017
14,767400,2017
17,772540,2018
...,...,...
71535,745400,2018
71539,773640,2018
71543,733530,2018
71546,610660,2018


Realizamos la fusión (merge) entre los dos DataFrames: df_reviews_sentiment y df_games_sentiment. La fusión se realiza en base a una columna común, en este caso, 'reviews_item_id' de df_reviews_sentiment y 'id' de df_games_sentiment.

In [140]:
df_sentiment = df_reviews_sentiment.merge(df_games_sentiment, left_on='reviews_item_id', right_on='id', how='left')

In [141]:
df_sentiment.head()

,user_id,reviews_item_id,sentiment_analysis,id,release_year
0,76561197970982479,1250,2,1250.0,2009
1,js41637,251610,2,NaN,NaN
2,evcentric,248820,2,248820.0,2013
3,doctr,250320,2,250320.0,2013
4,maplemage,211420,1,211420.0,2012


Agrupamos el DataFrame df_sentiment por 'release_year' (año de lanzamiento) y 'sentiment_analysis' (análisis de sentimiento). La función groupby se utiliza para agrupar las filas de acuerdo a estas dos columnas. Luego, usamos la función size() para contar las ocurrencias de cada combinación de año y sentimiento. El resultado es un DataFrame llamado sentiment_counts, donde las filas representan los años y las columnas representan los sentimientos. La función unstack(fill_value=0) reorganiza el DataFrame para que los sentimientos se conviertan en columnas, y los valores faltantes se rellenan con ceros.

In [142]:
# Agrupar por año y sentimiento, y contar las ocurrencias
sentiment_counts = df_sentiment.groupby(['release_year', 'sentiment_analysis']).size().unstack(fill_value=0)


In [147]:
# Agrupar por año y sentimiento, y contar las ocurrencias
sentiment_counts = df_sentiment.groupby(['release_year', 'sentiment_analysis']).size().unstack(fill_value=0)

# Restablecer el índice para obtener el DataFrame en su formato normal
sentiment_counts = sentiment_counts.reset_index()


In [149]:
sentiment_counts.index.name = None

In [151]:
sentiment_counts

sentiment_analysis,release_year,0,1,2
0,1989,0,0,1
1,1990,1,0,4
2,1991,0,0,1
3,1992,1,0,2
4,1993,1,1,3
5,1994,2,5,7
6,1995,2,3,10
7,1996,0,3,4
8,1997,6,10,31
9,1998,10,31,54


El DataFrame sentiment_counts proporciona un recuento detallado de reseñas de usuarios categorizadas por año y sentimiento.

Siguiendo la misma práctica que en los casos anteriores,guardamos los resultados en formato CSV, lo cual simplifica la gestión de datos durante el desarrollo de nuestra función 'sentiment_analysis( año : int )'. Además, almacenamos los DataFrames en formato Parquet para mejorar la estructura de los datos en la fase de implementación.

In [152]:
# Guardar en CSV
sentiment_counts.to_csv('df_sentiment_counts.csv', index=False, encoding='utf-8')
print(f'Se ha guardado el archivo df_sentiment_counts.csv en la misma carpeta.')

Se ha guardado el archivo df_sentiment_counts.csv en la misma carpeta.


In [153]:
# Guardar en Parquet
table = pa.Table.from_pandas(sentiment_counts)
pq.write_table(table, 'df_sentiment_counts.parquet')
print(f'Se ha guardado el archivo df_sentiment_counts.parquet en la misma carpeta.')

Se ha guardado el archivo df_sentiment_counts.parquet en la misma carpeta.
